NAEEM:
    - AUTOMATION
    - INTERPRETATION
    - FEATURE EXTRACTION

In [ ]:
import pyshark

In [ ]:
capture = pyshark.LiveCapture(interface='en0') # read from wifi interface.

# for reading from a .cap file, we can use the following line instead
# capture = pyshark.fileCapture('/path/to/mycapturefile.cap')

capture.sniff(timeout=60) # sniff sniff for a minute!

In [ ]:
capture
# should output something like <LiveCapture (X packets)>
# we can also output something like
capture[1]
# to see what type of packet it is! output should be like <TCP Packet>

In [ ]:
# We can sniff the packets again in the capture array to see what we have in detail
for packet in capture.sniff_continuously(packet_count=10):
    print('***** ARRIVED *****: ', packet)

In [4]:
from scapy.all import * # Packet manipulation
import pandas as pd # Pandas - Create and Manipulate DataFrames
import numpy as np # Math Stuff (don't worry only used for one line :] )
import binascii # Binary to Ascii 
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline

In [10]:
pcap = rdpcap('youtube.pcap')

In [11]:
pcap[0].show()

###[ Ethernet ]### 
  dst       = 33:33:00:00:00:16
  src       = 52:54:00:12:34:56
  type      = 0x86dd
###[ IPv6 ]### 
     version   = 6
     tc        = 0
     fl        = 0
     plen      = 76
     nh        = Hop-by-Hop Option Header
     hlim      = 1
     src       = ::
     dst       = ff02::16
###[ IPv6 Extension Header - Hop-by-Hop Options Header ]### 
        nh        = ICMPv6
        len       = 0
        autopad   = On
        \options   \
         |###[ Router Alert ]### 
         |  otype     = Router Alert [00: skip, 0: Don't change en-route]
         |  optlen    = 2
         |  value     = Datagram contains a MLD message
         |###[ PadN ]### 
         |  otype     = PadN [00: skip, 0: Don't change en-route]
         |  optlen    = 0
         |  optdata   = ''
###[ MLDv2 - Multicast Listener Report ]### 
           type      = MLD Report Version 2
           res       = 0
           cksum     = 0x34eb
           reserved  = 0
           records_number= 3
        

In [12]:
# Collect field names from IP/TCP/UDP (These will be columns in DF)
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]

dataframe_fields = ip_fields + ['time'] + tcp_fields + ['payload','payload_raw','payload_hex']

# Create blank DataFrame
df = pd.DataFrame(columns=dataframe_fields)
for packet in pcap[IP]:
    # Field array for each row of DataFrame
    field_values = []
    # Add all IP fields to dataframe
    for field in ip_fields:
        if field == 'options':
            # Retrieving number of options defined in IP Header
            field_values.append(len(packet[IP].fields[field]))
        else:
            field_values.append(packet[IP].fields[field])
    
    field_values.append(packet.time)
    
    layer_type = type(packet[IP].payload)
    for field in tcp_fields:
        try:
            if field == 'options':
                field_values.append(len(packet[layer_type].fields[field]))
            else:
                field_values.append(packet[layer_type].fields[field])
        except:
            field_values.append(None)
    
    # Append payload
    field_values.append(len(packet[layer_type].payload))
    field_values.append(packet[layer_type].payload.original)
    field_values.append(binascii.hexlify(packet[layer_type].payload.original))
    # Add row to DF
    df_append = pd.DataFrame([field_values], columns=dataframe_fields)
    df = pd.concat([df, df_append], axis=0)

# Reset Index
df = df.reset_index()
# Drop old index column
df = df.drop(columns="index")

In [17]:
df.head()

,version,ihl,tos,len,id,flags,frag,ttl,proto,chksum,...,dataofs,reserved,flags,window,chksum,urgptr,options,payload,payload_raw,payload_hex
0,4,5,0,282,0,,0,64,17,31188,...,None,None,None,None,34314,None,None,254,      �                 RT 4V            ...,0101060000000001000080000000000000000000000000...
1,4,5,16,576,0,,0,64,17,27804,...,None,None,None,None,47984,None,None,548,            \n \n     RT 4V          ...,020106000000000100000000000000000a00020f0a0002...
2,4,5,0,294,0,,0,64,17,31176,...,None,None,None,None,27249,None,None,266,      �                 RT 4V            ...,0101060000000002000080000000000000000000000000...
3,4,5,16,576,1,,0,64,17,27803,...,None,None,None,None,47215,None,None,548,            \n \n     RT 4V          ...,020106000000000200000000000000000a00020f0a0002...
4,4,5,0,62,3101,DF,0,63,17,6017,...,None,None,None,None,15309,None,None,34,�;        timeandroidcom    ,e53b010000010000000000000474696d6507616e64726f...
